<a href="https://colab.research.google.com/github/jaircampfens/Paper-Dynamic-Bayesian-Networks-/blob/main/Survey_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Survey Design

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import pandas as pd
from datetime import datetime
import os


SURVEY_TITLE = "Survey about Energy Transition Pathways for Winterthur"
SURVEY_INTRO = """
Please rate, on a scale from 1 to 10, how strong you think each relationship is
for the different years shown in the figure (2025, 2033, 2040).

1 = No impact, 10 = Very impactful.
"""

LIKERT_MIN_LABEL = "No impact"
LIKERT_MAX_LABEL = "Very impactful"

# Time steps / scenarios
time_steps = [
    {"id": "2025", "label": "2025 (Current state)"},
    {"id": "2033", "label": "2033 (Target: 1 ton CO₂)"},
    {"id": "2040", "label": "2040 (Target: net-zero)"},
]

# Relations grouped as in your network
relations_config = [
    {
        "group_id": "final",
        "name": "Impact on Final Node (Greenhouse Gas Emissions)",
        "description": "Impact of the energy use nodes on GHG Emissions.",
        "edges": [
            "Electricity Demand -> Greenhouse Gas Emissions",
            "Sustainable Mobility -> Greenhouse Gas Emissions",
            "Sustainable Heating -> Greenhouse Gas Emissions",
        ],
    },
    {
        "group_id": "energy_use",
        "name": "Impact on Energy Use Nodes",
        "description": "Impact of the technical nodes on Electricity Demand, Sustainable Mobility, Sustainable Heating.",
        "edges": [
            "PV -> Electricity Demand",
            "Bike Users -> Sustainable Mobility",
            "EV -> Sustainable Mobility",
            "Heat Pump -> Sustainable Heating",
            "District Heating -> Sustainable Heating",
            "Bio Gas -> Sustainable Heating",
        ],
    },
    {
        "group_id": "technical",
        "name": "Impact on Technical Nodes",
        "description": "Impact of information / social nodes on PV, EV, Heat Pump, District Heating, Bio Gas, Bike Users.",
        "edges": [
            "Information Facilities -> PV",
            "Information Facilities -> Bike Users",
            "Information Facilities -> EV",
            "Information Facilities -> Heat Pump",
            "Perceived Viability -> PV",
            "Perceived Viability -> EV",
            "Perceived Viability -> Heat Pump",
            "Perceived Viability -> District Heating",
            "Environmental Norm -> PV",
            "Environmental Norm -> Bio Gas",
        ],
    },
]

# Perspective & comments
perspective_options = ["Policy", "Academia", "Enterprise", "Civil Society", "Other"]


In [ ]:
likert_widgets = {}   # key: (time_id, edge)  -> slider

time_step_boxes = []

for ts in time_steps:
    ts_header = widgets.HTML(
        value=f"<h2>{ts['label']}</h2>"
    )

    group_boxes = []
    for group in relations_config:
        g_header = widgets.HTML(
            value=f"<h3>{group['name']}</h3><p>{group['description']}</p>"
        )

        q_boxes = []
        for edge in group["edges"]:
            # unique key for this year + edge
            key = (ts["id"], edge)

            slider = widgets.IntSlider(
                value=5,
                min=1,
                max=10,
                step=1,
                description="",
                continuous_update=False,
                style={'description_width': '0px'},
                layout=Layout(width='300px')
            )
            likert_widgets[key] = slider

            label = widgets.HTML(
                value=f"<b>{edge}</b><br>"
                      f"<small>{LIKERT_MIN_LABEL} (1) – {LIKERT_MAX_LABEL} (10)</small>"
            )

            q_box = VBox([label, slider], layout=Layout(margin="0 0 10px 0"))
            q_boxes.append(q_box)

        group_box = VBox(
            [g_header] + q_boxes,
            layout=Layout(border="1px solid #ccc", padding="10px", margin="10px 0")
        )
        group_boxes.append(group_box)

    ts_box = VBox(
        [ts_header] + group_boxes,
        layout=Layout(border="2px dotted #000", padding="10px", margin="15px 0")
    )
    time_step_boxes.append(ts_box)

# Perspective and comments
perspective_label = widgets.HTML("<h3>With which perspective do you identify the most?</h3>")
perspective_widget = widgets.Dropdown(
    options=perspective_options,
    value=perspective_options[0],
    description="Perspective:"
)

comments_label = widgets.HTML("<h3>Final comments or questions</h3>")
comments_widget = widgets.Textarea(
    placeholder="Any additional comments or questions about the project or the survey.",
    layout=Layout(width="100%", height="100px")
)


In [ ]:
CSV_PATH = "survey_responses.csv"

submit_button = widgets.Button(
    description="Submit survey",
    button_style="success",
    tooltip="Save your answers",
    layout=Layout(width="200px")
)

output = widgets.Output()

def on_submit_clicked(b):
    with output:
        output.clear_output()

        data = {}
        data["timestamp"] = datetime.now().isoformat(timespec="seconds")

        # add all sliders
        for (time_id, edge), widget in likert_widgets.items():
            col_name = f"{time_id}: {edge}"
            data[col_name] = widget.value

        # perspective & comments
        data["Perspective"] = perspective_widget.value
        data["Comments"] = comments_widget.value

        row_df = pd.DataFrame([data])

        if os.path.exists(CSV_PATH):
            existing = pd.read_csv(CSV_PATH)
            combined = pd.concat([existing, row_df], ignore_index=True)
            combined.to_csv(CSV_PATH, index=False)
        else:
            row_df.to_csv(CSV_PATH, index=False)

        print("✅ Thank you! Your response has been recorded.")
        print(f"Responses are stored in: {CSV_PATH}")

submit_button.on_click(on_submit_clicked)


In [ ]:
title_html = widgets.HTML(f"<h1>{SURVEY_TITLE}</h1>")
intro_html = widgets.HTML(f"<p>{SURVEY_INTRO}</p>")

survey_ui = VBox(
    [title_html, intro_html]
    + time_step_boxes
    + [
        perspective_label,
        perspective_widget,
        comments_label,
        comments_widget,
        submit_button,
        output,
    ],
    layout=Layout(width="900px")
)

survey_ui


In [ ]:
if os.path.exists(CSV_PATH):
    display(pd.read_csv(CSV_PATH))
else:
    print("No responses saved yet.")


No responses saved yet.
